# PPD: Programação com CUDA

Hélio - DC/UFSCar - 2023

# Estudo de caso: soma de matrizes

Neste estudo de casos, investigamos a realização de uma soma de 2 matrizes em GPU.

É claro que a ideia é realizar a soma de cada par de elementos em paralelo, criando um número de *threads* igual ao número de elementos a somar, o que corresponde à multiplicação do número de linhas pelo número de colunas.

Há outras questões a considerar, contudo. A primeira é como será o armazenamento das matrizes na memória da GPU.

Supondo que os dados seriam lidos de um arquivo, ou gerados de alguma outra forma em CPU, e não na própria GPU, é preciso alocar espaço na memória do dispositivo (*device*/GPU) e copiar os dados da RAM para esse espaço alocado na GPU.

Como organizar os dados das matrizes na memória da GPU? Comumente, alocar o espaço como uma sequência de linhas parece mais fácil.

Outra questão a considerar é como organizar as *threads* em blocos e os blocos em grade, e como usar esses identificadores para determinar a soma de quais elementos será calculada por cada *thread*.

Vejamos 2 exemplos, que exploram **blocos 2D e grade 2D** e **blocos 1D e grade 1D**.


# Bloco 2D e grade 2D

Nesta estratégia de particionamento do cálculos dos elementos, tanto o bloco de *threads* quanto a grade de blocos são organizados em estruturas 2D, utilizando os parâmetros .x e .y da estrutura dim3 (.z é mantido em 1).

Como se vê, a estratégia de ajustes dos índices ***i*** e ***j*** das *threads* é a mesma, que considera o identificador do bloco vezes o tamanho do bloco, em cada uma das dimensões (.x e .y).

Ao final, os valores i e j acabam sendo linearizados para o acesso à matriz, da forma tradicional (i * num_col + j).


In [ ]:
%%writefile 2dx2d.cu

#include <stdio.h>
#include <time.h>

#define DIM 10240


__global__ void somaMatrizes(float *A, float *B, float *C, int dim )
{
	unsigned idx = blockIdx.x * blockDim.x + threadIdx.x;
  unsigned idy = blockIdx.y * blockDim.y + threadIdx.y;

	if (idx < dim && idy < dim)
		C[idx * dim + idy] = A[idx * dim + idy] + B[idx * dim + idy];
}

int main()
{
  float *A, *B, *C;          // ponteiros para matrizes na RAM
  float *d_A, *d_B, *d_C;    // ponteiros para matrizes na GPU

  int n_elem = DIM;
  size_t tam_mat = n_elem * n_elem * sizeof(float);

  // Alocando matrizes na RAM como sequências de linhas
  A = (float *)malloc( tam_mat );
  B = (float *)malloc( tam_mat );
  C = (float *)malloc( tam_mat );

  // Atribuição de valores para as matrizes
  // É claro que, para gerar valores desta forma, poderíamos fazer isso na GPU, em paralelo :-)
  for(int i=0; i < n_elem * n_elem; i++) {
    A[i] = 0.1;
    B[i] = 0.2;
  }

  // Alocação de espaço para as matrizes na GPU
	cudaMalloc((void **)&d_A, tam_mat );
	cudaMalloc((void **)&d_B, tam_mat );
	cudaMalloc((void **)&d_C, tam_mat );

  // Cópia dos dados de A e B para d_A e d_B. Não precisa copiar C!
	cudaMemcpy(d_A, A, tam_mat, cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, B, tam_mat, cudaMemcpyHostToDevice);


  // Definição do bloco e da grade
  dim3 block;

  block.x = 16;
  block.y = 16;
  block.z = 1;

  dim3 grid;
  grid.x = (n_elem + block.x -1) / block.x;
  grid.y = (n_elem + block.y -1) / block.y;
  grid.z = 1;

  // Invocação do kernel
	somaMatrizes <<< grid, block >>> (d_A, d_B, d_C, n_elem );

  cudaError_t err = cudaGetLastError();
  if (err!= cudaSuccess) {
    printf("Erro: %s\n", cudaGetErrorString(err));
    exit(0);
  }

  // Copia dados resultantes da memória da GPU para a RAM
  // Operação de cópia é síncrona: só executará depois de concluída a operação anterior
  // na GPU (execução do kernel, neste caso) e só retornará depois de concluída a transferência
	cudaMemcpy(C, d_C, tam_mat, cudaMemcpyDeviceToHost);

  // nao precisa fazer a sincronizacao abaixo, já que memcpy é síncrona
  // cudaDeviceSynchronize();

  // Conferindo os valores produzidos
  float maxError = 0.0f;
  for (int i = 0; i < n_elem * n_elem; i++)
    maxError = max(maxError, abs(C[i]-0.3));
  printf("Max error: %f\n", maxError);

  // Libera memória na GPU
	cudaFree(d_A);
	cudaFree(d_B);
	cudaFree(d_C);

	// Libera memória - CPU / RAM
	free(A);
	free(B);
	free(C);

	return 0;
}


Writing 2dx2d.cu


In [ ]:
! if [ ! 2dx2d -nt 2dx2d.cu ]; then nvcc 2dx2d.cu -o 2dx2d -O3; fi
! nvprof --print-gpu-trace ./2dx2d
# ! ./2dx2d

==469== NVPROF is profiling process 469, command: ./2dx2d
Max error: 0.000000
==469== Profiling application: ./2dx2d
==469== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
423.69ms  87.500ms                    -               -         -         -         -  400.00MB  4.4643GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
511.41ms  86.516ms                    -               -         -         -         -  400.00MB  4.5151GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
597.93ms  31.541ms          (320 320 1)       (32 32 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  somaMatrizes(float*, float*, float*, int) [116]
629.48ms  264.33ms                    -               -         -         -         - 

# Bloco 1D e grade 1D

Nesta estratégia de particionamento do cálculos dos elementos, tanto o bloco de *threads* quanto a grade de blocos são organizados em estruturas com 1 dimensão.

Como as matrizes já estão linearizadas, é possível fazer um mapeamento direto entre o índice da *thread* e o índice das matrizes.


In [ ]:
%%writefile 1dx1d.cu

#include <stdio.h>
#include <time.h>

#define DIM 10240


__global__ void somaMatrizes(float *A, float *B, float *C, int dim )
{
	unsigned idx = blockIdx.x * blockDim.x + threadIdx.x;

	if (idx < dim * dim)
		C[idx] = A[idx] + B[idx];
}


int main()
{
  float *A, *B, *C;          // ponteiros para matrizes na RAM
  float *d_A, *d_B, *d_C;    // ponteiros para matrizes na GPU

  int n_elem = DIM;
  size_t tam_mat = n_elem * n_elem * sizeof(float);

  // Alocando matrizes na RAM como sequências de linhas
  A = (float *)malloc( tam_mat );
  B = (float *)malloc( tam_mat );
  C = (float *)malloc( tam_mat );

  // Atribuição de valores para as matrizes
  // É claro que, para gerar valores desta forma, poderíamos fazer isso na GPU, em paralelo :-)
  for(int i=0; i < n_elem * n_elem; i++) {
    A[i] = 0.1;
    B[i] = 0.2;
  }

  // Alocação de espaço para as matrizes na GPU
	cudaMalloc((void **)&d_A, tam_mat );
	cudaMalloc((void **)&d_B, tam_mat );
	cudaMalloc((void **)&d_C, tam_mat );

  // Cópia dos dados de A e B para d_A e d_B. Não precisa copiar C!
	cudaMemcpy(d_A, A, tam_mat, cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, B, tam_mat, cudaMemcpyHostToDevice);


  // Definição do bloco e da grade
  // dim3 block;
  // block.x = 32; block.y = 32; block.z = 1;

  // dim3 grid;
  // grid.x = (n_elem + block.x -1) / block.x;  grid.y = (n_elem + block.y -1) / block.y; grid.z = 1;

  int block = 32;
  int grid = (n_elem * n_elem + block -1) / block;

  // Invocação do kernel
	somaMatrizes <<< grid, block >>> (d_A, d_B, d_C, n_elem );

  cudaError_t err = cudaGetLastError();
  if (err!= cudaSuccess) {
    printf("Erro: %s\n", cudaGetErrorString(err));
    exit(0);
  }

  // Copia dados resultantes da memória da GPU para a RAM
  // Operação de cópia é síncrona: só executará depois de concluída a operação anterior
  // na GPU (execução do kernel, neste caso) e só retornará depois de concluída a transferência
	cudaMemcpy(C, d_C, tam_mat, cudaMemcpyDeviceToHost);

  // nao precisa fazer a sincronizacao abaixo, já que memcpy é síncrona
  // cudaDeviceSynchronize();

  // Conferindo os valores produzidos
  float maxError = 0.0f;
  for (int i = 0; i < n_elem * n_elem; i++)
    maxError = max(maxError, abs(C[i]-0.3));
  printf("Max error: %f\n", maxError);

  // Libera memória na GPU
	cudaFree(d_A);
	cudaFree(d_B);
	cudaFree(d_C);

	// Libera memória - CPU / RAM
	free(A);
	free(B);
	free(C);

	return 0;
}


Writing 1dx1d.cu


In [ ]:
! if [ ! 1dx1d -nt 1dx1d.cu ]; then nvcc 1dx1d.cu -o 1dx1d -O3; fi
! nvprof --print-gpu-trace ./1dx1d
! echo
# ! nvprof ./1dx1d
# ! ./1dx1d

==1277== NVPROF is profiling process 1277, command: ./1dx1d
Max error: 0.000000
==1277== Profiling application: ./1dx1d
==1277== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
413.50ms  85.487ms                    -               -         -         -         -  400.00MB  4.5694GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
499.22ms  88.854ms                    -               -         -         -         -  400.00MB  4.3962GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
588.08ms  11.450ms        (3276800 1 1)        (32 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  somaMatrizes(float*, float*, float*, int) [116]
599.54ms  257.86ms                    -               -         -         -       

Será que a organização das *threads* e dos blocos tem algum impacto no desempenho da execução do *kernel*?

Um aspecto a considerar é que o código da função do *kernel* 2dx2d tem mais instruções para o cálculo dos índices do que a versão 1dx1d:

```c
{ // 2dx2d
  unsigned idx = blockIdx.x * blockDim.x + threadIdx.x;
  unsigned idy = blockIdx.y * blockDim.y + threadIdx.y;
  if (idx < dim && idy < dim)
    C[idx * dim + idy] = A[idx * dim + idy] + B[idx * dim + idy];
}
```

```c
{ // 1dx1d
  unsigned idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx < dim * dim)
    C[idx] = A[idx] + B[idx];
}
```


In [ ]:
! echo Usando 2d x 2d:
! nvprof --print-gpu-trace ./2dx2d 2>&1 | grep somaMatrizes
! echo
! echo Usando 1d x 1d:
! nvprof --print-gpu-trace ./1dx1d 2>&1 | grep somaMatrizes

Usando 2d x 2d:
591.46ms  31.546ms          (320 320 1)       (32 32 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  somaMatrizes(float*, float*, float*, int) [116]

Usando 1d x 1d:
600.28ms  11.455ms        (3276800 1 1)        (32 1 1)        16        0B        0B         -           -           -           -     Tesla T4 (0)         1         7  somaMatrizes(float*, float*, float*, int) [116]
